In [ ]:
import logging

from clearml import Task
from clearml.automation import (
    DiscreteParameterRange, HyperParameterOptimizer, RandomSearch,
    UniformIntegerParameterRange, UniformParameterRange)

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# Pytorch
%env CLEARML_API_ACCESS_KEY=API_KEY
%env CLEARML_API_SECRET_KEY=API_SECRET

In [ ]:
# trying to load Bayesian optimizer package
try:
    from clearml.automation.optuna import OptimizerOptuna  # noqa
    aSearchStrategy = OptimizerOptuna
except ImportError as ex:
    try:
        from clearml.automation.hpbandster import OptimizerBOHB  # noqa
        aSearchStrategy = OptimizerBOHB
    except ImportError as ex:
        logging.getLogger().warning(
            'Apologies, it seems you do not have \'optuna\' or \'hpbandster\' installed, '
            'we will be using RandomSearch strategy instead')
        aSearchStrategy = RandomSearch


def job_complete_callback(
    job_id,                 # type: str
    objective_value,        # type: float
    objective_iteration,    # type: int
    job_parameters,         # type: dict
    top_performance_job_id  # type: str
):
    print('Job completed!', job_id, objective_value, objective_iteration, job_parameters)
    if job_id == top_performance_job_id:
        print('WOOT WOOT we broke the record! Objective reached {}'.format(objective_value))

In [ ]:
# Connecting ClearML with the current process,
# from here on everything is logged automatically
task = Task.init(project_name='ImageNetTE',
                 task_name='ImageNetTE - EfficientNetV2-S - Image Sizes',
                 task_type=Task.TaskTypes.optimizer,
                 reuse_last_task_id=False)

# Change this to change the task on which you base the hyper parameter tuning
template_id = "78bb69f6f721430ab06c76fcd7741830"

In [ ]:
# These are the hyper parameters which are randomly optimized
hyper_parameters = [
    DiscreteParameterRange('Stage Params/dropout_min', values=[0.1, 0.2]),
    DiscreteParameterRange('Stage Params/dropout_max', values=[ 0.3, 0.5, 0.7]),
    #DiscreteParameterRange('Stage Params/image_size_min', values=[128, 200]),
    #DiscreteParameterRange('Stage Params/image_size_max', values=[200, 300, 400]),
    DiscreteParameterRange('Stage Params/epochs', values=[20]),
    #DiscreteParameterRange('Stage Params/epochs', values=[20, 40, 60]),
]

In [ ]:
# Set default queue name for the Training tasks themselves.
# later can be overridden in the UI
execution_queue = 'default'

# Example use case:
an_optimizer = HyperParameterOptimizer(
    # This is the experiment we want to optimize
    base_task_id=template_id,
    # here we define the hyper-parameters to optimize
    # Notice: The parameter name should exactly match what you see in the UI: <section_name>/<parameter>
    # For Example, here we see in the base experiment a section Named: "General"
    # under it a parameter named "batch_size", this becomes "General/batch_size"
    # If you have `argparse` for example, then arguments will appear under the "Args" section,
    # and you should instead pass "Args/batch_size"
    hyper_parameters=hyper_parameters,
    # this is the objective metric we want to maximize/minimize
    objective_metric_title='Accuracy',
    objective_metric_series='Stage 4: validation',
    # now we decide if we want to maximize it or minimize it (accuracy we maximize)
    objective_metric_sign='max',
    # let us limit the number of concurrent experiments,
    # this in turn will make sure we do dont bombard the scheduler with experiments.
    # if we have an auto-scaler connected, this, by proxy, will limit the number of machine
    max_number_of_concurrent_tasks=1,
    # this is the optimizer class (actually doing the optimization)
    # Currently, we can choose from GridSearch, RandomSearch or OptimizerBOHB (Bayesian optimization Hyper-Band)
    # more are coming soon...
    optimizer_class=aSearchStrategy,
    # Select an execution queue to schedule the experiments for execution
    execution_queue=execution_queue,
    # If specified all Tasks created by the HPO process will be created under the `spawned_project` project
    spawn_project=None,
    # If specified only the top K performing Tasks will be kept, the others will be automatically archived
    save_top_k_tasks_only=3,
    # Optional: Limit the execution time of a single experiment, in minutes.
    # (this is optional, and if using  OptimizerBOHB, it is ignored)
    time_limit_per_job=200.,
    # Check the experiments every 60 seconds
    pool_period_min=1,
)

In [ ]:
# report every 60 seconds
an_optimizer.set_report_period(1)
# start the optimization process, callback function to be called every time an experiment is completed
# this function returns immediately
an_optimizer.start(job_complete_callback=job_complete_callback)
# You can also use the line below instead to run all the optimizer tasks locally, without using queues or agent
# an_optimizer.start_locally(job_complete_callback=job_complete_callback)
# set the time limit for the optimization process (2 hours)
an_optimizer.set_time_limit(in_minutes=1200.0)
# wait until process is done (notice we are controlling the optimization process in the background)
an_optimizer.wait()
# optimization is completed, print the top performing experiments id
top_exp = an_optimizer.get_top_experiments(top_k=3)
print([t.id for t in top_exp])
# make sure background optimization stopped
an_optimizer.stop()

print('We are done, good bye')
task.close()